In [72]:
import pandas as p
import numpy as np
import itertools
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import re

In [64]:
def getCleanTweet(text):
    tok = WordPunctTokenizer()
    user_pattern = '@[A-Za-z0-9_]+'
    http_pattern = 'https?://[^ ]+'
    www_pattern = 'www.[^ ]+'
    combined_pattern = '|'.join((user_pattern, http_pattern, www_pattern))
    negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                    "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                    "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                    "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                    "mustn't":"must not"}
    neg_pattern = re.compile('\b(' + '|'.join(negations_dic.keys()) + ')\b')
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pattern, '', bom_removed)
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], stripped)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled).lower()
    cleaned = (" ".join(x for x in tok.tokenize(letters_only) if len(x) > 1)).strip()
    cleaned = ''.join(k + k if sum(1 for i in g) > 1 else k for k, g in itertools.groupby(cleaned))
    return cleaned

In [67]:
data = p.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin-1")
data['clean_text'] = [getCleanTweet(text) for text in data['text']]
data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)
data.info()

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:314: UserWarning: "b' i just received my G8 viola exam.. and its... well... .. disappointing.. :\\..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:314: UserWarning: "b'E3 ON PLAYSTATION HOME IN ABOUT AN HOUR!!!!!!!!!! \\../  \\../'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   target      1600000 non-null  int64 
 1   ids         1600000 non-null  int64 
 2   date        1600000 non-null  object
 3   flag        1600000 non-null  object
 4   user        1600000 non-null  object
 5   text        1600000 non-null  object
 6   clean_text  1600000 non-null  object
dtypes: int64(2), object(5)
memory usage: 85.4+ MB


In [71]:
%store data

Stored 'data' (DataFrame)
